## La Liga Big 3 (2014–2025): Seasonal Goals & Expected Goals (xG) Comparison
- Real Madrid, Barcelona, and Atletico Madrid — season-by-season trends in Goals For (GF) and Expected Goals For (xGF), with simple interpretations.

### Step A. Imports & Paths(임포트와 경로)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

df = pd.read_csv('la_liga_2014-2025_match_info.csv')

print(df.head())

   match_id                 date home_team            away_team  home_goals  \
0      5826  2014-08-23 18:00:00    Malaga        Athletic Club           1   
1      5827  2014-08-23 20:00:00   Sevilla             Valencia           1   
2      5828  2014-08-23 20:00:00   Granada  Deportivo La Coruna           2   
3      5829  2014-08-23 22:00:00   Almeria             Espanyol           1   
4      5830  2014-08-24 18:00:00     Eibar        Real Sociedad           1   

   away_goals  home_xg  away_xg  home_shots  away_shots  home_sot  away_sot  \
0           0     1.32     1.14          12          12         3         5   
1           1     1.17     1.75          12          11         3         1   
2           1     0.55     0.38          10           8         3         1   
3           1     0.98     0.40          19          12         6         2   
4           0     0.47     0.98          12          19         5         4   

   home_deep  away_deep  home_ppda  away_ppda  hom

In [2]:
# Step A — Imports & Paths
# EN: Prepare libraries and set the CSV path.
# KO: 라이브러리를 불러오고 CSV 파일 경로를 설정합니다.
# Columns used: (none yet / 아직 없음)


### Step B. Load & Create Season Labels(로드 & 시즌 라벨 생성)

In [3]:
df = pd.read_csv('la_liga_2014-2025_match_info.csv')
df["date"] = pd.to_datetime(df["date"], errors="raise")

# Create 'season' if missing (La Liga runs Aug–May)
def make_season(y, m): 
    return f"{y}-{y+1}" if m >= 8 else f"{y-1}-{y}"

if "season" not in df.columns:
    df["season"] = df.apply(lambda r: make_season(r["date"].year, r["date"].month), axis=1)

print(df[["date","home_team","away_team","home_goals","away_goals","home_xg","away_xg"]].head())


                 date home_team            away_team  home_goals  away_goals  \
0 2014-08-23 18:00:00    Malaga        Athletic Club           1           0   
1 2014-08-23 20:00:00   Sevilla             Valencia           1           1   
2 2014-08-23 20:00:00   Granada  Deportivo La Coruna           2           1   
3 2014-08-23 22:00:00   Almeria             Espanyol           1           1   
4 2014-08-24 18:00:00     Eibar        Real Sociedad           1           0   

   home_xg  away_xg  
0     1.32     1.14  
1     1.17     1.75  
2     0.55     0.38  
3     0.98     0.40  
4     0.47     0.98  


In [4]:
# Step B — Load & Season Labels
# EN: Load matches and create a 'season' label (Aug–May) from the match date if missing.
# KO: 경기 데이터를 로드하고, 필요 시 날짜 기준(8~5월)으로 'season' 라벨을 생성합니다.
# Columns used: date (parse to datetime); if season missing, build from date.
# 사용 컬럼: date(날짜 파싱); season(없으면 date로 생성)


### Step C. Build team-match rows (Home & Away to one “team” table) / 팀-경기 테이블 만들기(홈/원정 → 팀 기준)

In [5]:
# Convert each match into two rows: one for home team, one for away team
home = df[["date","season","home_team","home_goals","away_goals","home_xg","away_xg"]].copy()
home.columns = ["date","season","team","gf","ga","xgf","xga"]

away = df[["date","season","away_team","away_goals","home_goals","away_xg","home_xg"]].copy()
away.columns = ["date","season","team","gf","ga","xgf","xga"]

tm = pd.concat([home, away], ignore_index=True)
tm.sort_values(["team","date"], inplace=True)

# Points per match (3/1/0) for later (optional)
tm["pts"] = np.where(tm["gf"]>tm["ga"], 3, np.where(tm["gf"]==tm["ga"], 1, 0))

print(tm.head())


                    date     season    team  gf  ga   xgf   xga  pts
4947 2016-08-22 00:15:00  2016-2017  Alaves   1   1  0.03  4.18    1
777  2016-08-28 20:15:00  2016-2017  Alaves   0   0  1.12  0.28    1
4965 2016-09-10 22:30:00  2016-2017  Alaves   2   1  0.29  1.35    3
799  2016-09-19 22:45:00  2016-2017  Alaves   0   0  0.46  0.67    1
4989 2016-09-23 00:00:00  2016-2017  Alaves   1   2  0.26  1.42    0


In [6]:
# Step C — Team-Match Table (Home/Away -> Team rows)
# EN: Convert each match into two team rows (home team & away team) with GF/GA and xGF/xGA.
#     Also compute points per match (3 win / 1 draw / 0 loss) for later use.
# KO: 한 경기를 홈/원정 팀 기준 두 행으로 변환하고, 득점/실점(GF/GA)과 기대득점/실점(xGF/xGA)을 만듭니다.
#     이후 활용을 위해 경기당 승점(승3/무1/패0)도 계산합니다.
# Columns used: date, season, home_team, away_team, home_goals, away_goals, home_xg, away_xg
# Derived: team, gf, ga, xgf, xga, pts
# 사용 컬럼: 위와 동일 / 파생: team, gf, ga, xgf, xga, pts

### Step D. Filter the “Big 3” & Aggregate by Season (3강 필터 & 시즌별 집계)

In [7]:
# Team name robustness (handles minor variants)
big3_alias = {
    "Real Madrid": ["Real Madrid"],
    "Barcelona":   ["Barcelona","FC Barcelona"],
    "Atletico Madrid": ["Atletico Madrid","Atlético Madrid"]
}

def is_big3(name):
    return any(name in v for v in big3_alias.values()) or \
           any(any(alias==name for alias in v) for v in big3_alias.values())

# Safer: map to canonical names
def canonical(name):
    for k, aliases in big3_alias.items():
        if name in aliases:
            return k
    return name

tm["team_std"] = tm["team"].apply(canonical)
big3 = tm[tm["team_std"].isin(["Real Madrid","Barcelona","Atletico Madrid"])].copy()

# Season-level per-team averages
season_team = (big3
               .groupby(["season","team_std"])
               .agg(
                   matches=("team_std","size"),
                   GF=("gf","sum"), GA=("ga","sum"),
                   xGF=("xgf","sum"), xGA=("xga","sum"),
                   Pts=("pts","sum")
               )
               .reset_index())

# Per-match metrics
season_team["GF_per_match"]  = season_team["GF"]/season_team["matches"]
season_team["GA_per_match"]  = season_team["GA"]/season_team["matches"]
season_team["xGF_per_match"] = season_team["xGF"]/season_team["matches"]
season_team["xGA_per_match"] = season_team["xGA"]/season_team["matches"]
season_team["PPG"]           = season_team["Pts"]/season_team["matches"]  # points per game

season_team.head()


,season,team_std,matches,GF,GA,xGF,xGA,Pts,GF_per_match,GA_per_match,xGF_per_match,xGA_per_match,PPG
0,2014-2015,Atletico Madrid,38,67,29,57.05,29.08,78,1.763158,0.763158,1.501316,0.765263,2.052632
1,2014-2015,Barcelona,38,110,21,102.98,28.47,94,2.894737,0.552632,2.710000,0.749211,2.473684
2,2014-2015,Real Madrid,38,118,38,95.76,42.58,92,3.105263,1.000000,2.520000,1.120526,2.421053
3,2015-2016,Atletico Madrid,38,63,18,54.94,27.79,88,1.657895,0.473684,1.445789,0.731316,2.315789
4,2015-2016,Barcelona,38,112,29,113.59,34.05,91,2.947368,0.763158,2.989211,0.896053,2.394737


In [8]:
# Step D — Filter Big 3 & Aggregate by Season
# EN: Keep only Real Madrid, Barcelona, Atletico Madrid (canonical names) and aggregate by season.
#     Sum totals (GF/GA/xGF/xGA/points) and compute per-match metrics.
# KO: 레알 마드리드, 바르셀로나, 아틀레티코 마드리드만 남기고 시즌별로 집계합니다.
#     누계(GF/GA/xGF/xGA/승점)와 경기당 지표를 계산합니다.
# Columns used: season, team (mapped to team_std), gf, ga, xgf, xga, pts
# Derived: matches, GF_per_match, GA_per_match, xGF_per_match, xGA_per_match, PPG
# 사용 컬럼/파생: 위와 동일

### Step E. Plot 1: Seasonal GF per match (line) / 그래프 1: 시즌별 경기당 득점(GF)

In [9]:
import plotly.express as px

# EN: Interactive line with hover (Season, Team, GF per match)
# KO: 인터랙티브 라인 (시즌, 팀, 경기당 득점 표시)
gf_long = season_team[["season","team_std","GF_per_match"]].copy()

fig_gf = px.line(
    gf_long, x="season", y="GF_per_match", color="team_std",
    markers=True, title="Seasonal Goals For per Match — Big 3",
    labels={"season":"Season","GF_per_match":"GF per match","team_std":"Team"},
    hover_data={"season":True,"team_std":True,"GF_per_match":":.2f"}
)
fig_gf.update_layout(legend_title_text="Team", xaxis_tickangle=45)
fig_gf.show()


In [10]:
# Step E — Plot: Seasonal GF per Match (Big 3)
# EN: Line chart of goals for per match by season for the Big 3.
# KO: 3강의 시즌별 경기당 득점(GF)을 라인 차트로 비교합니다.
# Columns used: season_team['season','team_std','GF_per_match']
# 사용 컬럼: season_team['season','team_std','GF_per_match']

- Barcelona peaks around 2015–2017, then drops sharply by 2018–2020, and rises again by 2024–2025.
- Real Madrid dips around 2018–2021, then recovers 2022–2024; slight pullback in 2024–2025.
- Atletico Madrid sits lower than the other two most seasons, with a steady uptick from 2021–2024.

- 바르셀로나는 2015–2017 정점 → 2018–2020 급락 → 2024–2025 재상승.
- 레알 마드리드는 2018–2021 약세 → 2022–2024 회복, 2024–2025 소폭 하락.
- 아틀레티코는 대체로 두 팀보다 낮고, 2021–2024 완만한 상승.

### Step F. Plot 2: Seasonal xGF per match (line) / 그래프 2: 시즌별 경기당 기대득점(xGF)

In [11]:
# EN: Interactive line for expected goals (xGF) per match.
# KO: 경기당 기대득점(xGF) 인터랙티브 라인.
xgf_long = season_team[["season","team_std","xGF_per_match"]].copy()

fig_xgf = px.line(
    xgf_long, x="season", y="xGF_per_match", color="team_std",
    markers=True, title="Seasonal Expected Goals For (xGF) per Match — Big 3",
    labels={"season":"Season","xGF_per_match":"xGF per match","team_std":"Team"},
    hover_data={"season":True,"team_std":True,"xGF_per_match":":.2f"}
)
fig_xgf.update_layout(legend_title_text="Team", xaxis_tickangle=45)
fig_xgf.show()


In [12]:
# Step F — Plot: Seasonal xGF per Match (Big 3)
# EN: Line chart of expected goals for per match (xGF) by season for the Big 3.
# KO: 3강의 시즌별 경기당 기대득점(xGF)을 라인 차트로 비교합니다.
# Columns used: season_team['season','team_std','xGF_per_match']
# 사용 컬럼: season_team['season','team_std','xGF_per_match']

- Barcelona leads xGF in mid-2010s, then declines to 2019–2021, rebounds afterwards.
- Real Madrid shows a trough near 2020–2021, then climbs again 2021–2025.
- Atletico has the lowest xGF overall, but trends upward from 2021–2025.


- 바르셀로나는 2010년대 중반 xGF 선두 → 2019–2021 하락 → 이후 회복.
- 레알 마드리드는 2020–2021 저점 → 2021–2025 재상승.
- 아틀레티코는 xGF가 전반적으로 가장 낮지만 2021–2025 상승 추세.